In [31]:
import xarray as xr
import dask
import itertools
import dask.bag as db
import numpy as np
import xclim.analog

import preprocess_obs
import myfuncs

## Read and stack data

In [2]:
cafe_file = '/g/data/xv83/dbi599/precip_cafe-c5-d60-pX-f6_19901101-19931101_3650D_cafe-grid-TAS-POINT.zarr.zip'
ds_cafe = xr.open_zarr(cafe_file)
ds_cafe                

<xarray.Dataset>
Dimensions:    (ensemble: 96, init_date: 4, lead_time: 3650)
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) datetime64[ns] 1990-11-01 1991-11-01 ... 1993-11-01
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3644 3645 3646 3647 3648 3649
    time       (lead_time, init_date) datetime64[ns] dask.array<chunksize=(3650, 4), meta=np.ndarray>
Data variables:
    precip     (init_date, lead_time, ensemble) float64 dask.array<chunksize=(4, 3650, 96), meta=np.ndarray>
Attributes:
    history:  Tue Jun 08 16:44:46 2021: /g/data/e14/dbi599/miniconda3/envs/un...

In [3]:
ds_cafe['init_date']

<xarray.DataArray 'init_date' (init_date: 4)>
array(['1990-11-01T00:00:00.000000000', '1991-11-01T00:00:00.000000000',
       '1992-11-01T00:00:00.000000000', '1993-11-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * init_date  (init_date) datetime64[ns] 1990-11-01 1991-11-01 ... 1993-11-01

In [19]:
#def resample_forecasts_monthly(ds, n_months, end_month, method='sum', 
#                               lead_time_func='max'):
#    """ Resample forecasts to a set of months for each year"""
#    def _resample_grouped_monthly_forecasts(ds, method):
#        month = np.unique(ds.init_date.dt.month)
#        assert len(month) == 1
#        start_month = (end_month - (n_months-1)) % 12
#        i_start = (start_month - month[0]) % 12
#        rs = ds.isel(lead_time=slice(i_start, None)).coarsen(
#            lead_time=n_months, boundary='trim',
#            coord_func={'lead_time': lead_time_func, 
#                        'time': 'max'})
#        if method == 'sum':
#            return rs.sum()
#        elif method == 'mean':
#            return rs.mean()
#        else:
#            raise InputError('Unrecognised method')
#    
#    rs = ds.groupby('init_date.month').map(
#        _resample_grouped_monthly_forecasts, method=method)
#    rs['time'].attrs = ds['time'].attrs
#    return rs

In [33]:
fcst_stacked = ds_cafe.stack({'sample': ['ensemble', 'init_date', 'lead_time']})
fcst_stacked

<xarray.Dataset>
Dimensions:    (sample: 1401600)
Coordinates:
    time       (sample) datetime64[ns] dask.array<chunksize=(1401600,), meta=np.ndarray>
  * sample     (sample) MultiIndex
  - ensemble   (sample) int64 1 1 1 1 1 1 1 1 1 1 ... 96 96 96 96 96 96 96 96 96
  - init_date  (sample) datetime64[ns] 1990-11-01 1990-11-01 ... 1993-11-01
  - lead_time  (sample) int64 0 1 2 3 4 5 6 ... 3644 3645 3646 3647 3648 3649
Data variables:
    precip     (sample) float64 dask.array<chunksize=(1401600,), meta=np.ndarray>
Attributes:
    history:  Tue Jun 08 16:44:46 2021: /g/data/e14/dbi599/miniconda3/envs/un...

In [5]:
#ds_stacked = ds_stacked.assign_coords({'time': ds_stacked['time']})

In [21]:
awap_file = '/g/data/xv83/ds0092/data/csiro-dcfp-csiro-awap/rain_day_19000101-20201202_cafe-grid.zarr/'

In [23]:
da_awap = myfuncs.open_file(awap_file, 'precip', no_leap_days=True,
                            region='TAS-POINT', units='mm/day')
da_awap = da_awap.sel(time=slice('1990-01-01', '1993-12-31'))
da_awap

<xarray.DataArray 'precip' (time: 1460)>
dask.array<getitem, shape=(1460,), dtype=float64, chunksize=(1460,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 1990-01-01 09:00:00 ... 1993-12-31 09:00:00
Attributes:
    analysis_time:                 2010-10-23 14:49:46 UTC
    analysis_version_number:       3.01
    cell_methods:                  time: sum
    frequency:                     daily
    grid_mapping:                  crs
    length_scale_for_analysis:     80.0
    long_name:                     Daily precipitation
    number_of_stations_reporting:  2757
    source:                        Interpolated surface observations
    standard_name:                 lwe_thickness_of_precipitation_amount
    units:                         mm/day
    valid_range:                   [-1.0, 100000.0]

In [24]:
obs_stacked = da_awap.stack(sample=['time'])

In [25]:
obs_stacked

<xarray.DataArray 'precip' (sample: 1460)>
dask.array<getitem, shape=(1460,), dtype=float64, chunksize=(1460,), chunktype=numpy.ndarray>
Coordinates:
  * sample   (sample) MultiIndex
  - time     (sample) object 1990-01-01 09:00:00 ... 1993-12-31 09:00:00
Attributes:
    analysis_time:                 2010-10-23 14:49:46 UTC
    analysis_version_number:       3.01
    cell_methods:                  time: sum
    frequency:                     daily
    grid_mapping:                  crs
    length_scale_for_analysis:     80.0
    long_name:                     Daily precipitation
    number_of_stations_reporting:  2757
    source:                        Interpolated surface observations
    standard_name:                 lwe_thickness_of_precipitation_amount
    units:                         mm/day
    valid_range:                   [-1.0, 100000.0]

## Fidelity test

In [29]:
dir(xclim)

['__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__email__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'atmos',
 'build_indicator_module_from_yaml',
 'core',
 'data',
 'f',
 'indicators',
 'indices',
 'land',
 'path',
 'seaIce',
 'set_options',
 'units']

In [30]:
xclim.__version__

'0.27.0'

In [34]:
xclim.analog.spatial_analogs(target=obs_stacked,
                             candidates=fcst_stacked,
                             dist_dim='sample',
                             method='kolmogorov_smirnov')

AttributeError: 'DataArray' object has no attribute 'data_vars'

I think the inputs here need to be xarray.Datasets with a data_vars attribute that is 'precip' in this case (could be two variables for multivariate KS test) 

In [19]:
def unseen_univariate_moment_fidelity(fcst, obsv, period, alpha=5, n_repeats=1000, n_block=1, by_lead=True):
    """ Run unseen fidelity test
        if alpha == None, returns two Datasets
            n_repeats samples of the moments from the model
            moments from the observations
        else returns single Dataset with
            0 where obsv falls between alpha/2 % and 100-alpha/2 % of the bootstrapped model distribution
            -1 where the obsv falls above 100-alpha/2 %
            1 where the obsv falls below alpha/2 %
    """
        
    fcst_period_stacked = stack_super_ensemble(fcst, period, 
                                               super_ensemble_dims=('init_date','ensemble') 
                                               if by_lead else ('init_date','ensemble', 'lead_time'))
    obsv_period = obsv.where((obsv.time >= period[0]) & (obsv.time <= period[1]), drop=True)
    
    fcst_moments = n_random_resamples(fcst_period_stacked, 
                                      samples={'super_ensemble': (len(obsv_period['time']), n_block)}, 
                                      n_repeats=n_repeats,
                                      function=get_first_four_moments,
                                      function_kwargs={'dim':'super_ensemble'})

    obsv_moments = get_first_four_moments(obsv_period, 'time')
    
    if alpha:
        fcst_moments_upper = fcst_moments.quantile(1-alpha/200, dim='k')
        fcst_moments_lower = fcst_moments.quantile(alpha/200, dim='k')

        not_nan = (obsv_moments * fcst_moments_lower * fcst_moments_upper).notnull()
        above_min = obsv_moments >= fcst_moments_lower
        below_max = obsv_moments <= fcst_moments_upper

        passed = above_min & below_max
        too_low = (above_min == False) & (passed == False) 
        too_high = (below_max == False) & (passed == False)

        return (xr.where(passed, 2, 0) + xr.where(too_low, 3, 0) + xr.where(too_high, 1, 0) - 2).where(not_nan)
    else:
        return fcst_moments, obsv_moments

In [20]:
def get_first_four_moments(da, dim):
    """ Return the mean, std, skewness and kurtosis along dim """
    
    ds = da.mean(dim).to_dataset(name='mean')
    daf = da - da.mean(dim)
    ds['std'] = da.std(dim)
    ds['skew'] = ((daf ** 3).mean(dim) / ((daf ** 2).mean(dim) ** (3/2)))
    ds['kurt'] = ((daf ** 4).mean(dim) / ((daf ** 2).mean(dim) ** (2)))
    
    return ds

In [21]:
awap_moments = get_first_four_moments(da_awap, 'time')

In [29]:
awap_moments['mean'].data.shape

()

In [10]:
def random_resample(*objects, samples,
                    function=None, function_kwargs=None, replace=True):
    """
        Randomly resample from provided xarray objects and return the results of the subsampled dataset passed through \
        a provided function
                
        Parameters
        ----------
        *objects : xarray DataArray or Dataset
            Objects containing data to be resampled. The coordinates of the first object are used for resampling and the \
            same resampling is applied to all objects
        samples : dictionary
            Dictionary containing the dimensions to subsample, the number of samples and the continuous block size \
            within the sample. Of the form {'dim1': (n_samples, block_size), 'dim2': (n_samples, block_size)}. The first \
            object in objects must contain all dimensions listed in samples, but subsequent objects need not.
        function : function object, optional
            Function to reduced the subsampled data
        function_kwargs : dictionary, optional
            Keyword arguments to provide to function
        replace : boolean, optional
            Whether the sample is with or without replacement
                
        Returns
        -------
        sample : xarray DataArray or Dataset
            Array containing the results of passing the subsampled data through function
    """
    objects_sub = [obj.copy() for obj in objects]
    for dimension, (n_samples, block_size) in samples.items():
        n_blocks = int(n_samples / block_size)
        n_samples = n_blocks * block_size
        random_samples = [slice(x,x+block_size) for x in np.random.choice(len(objects_sub[0][dimension])-block_size+1, 
                                                                          size=n_blocks,
                                                                          replace=replace)]
        objects_sub = [xr.concat([obj.isel({dimension: random_sample}) for random_sample in random_samples],
                                 dim=dimension) 
                       if dimension in obj.dims else obj 
                       for obj in objects_sub]
    
    if function:
        if function_kwargs:
            res = tuple([function(obj, **function_kwargs) for obj in objects_sub])
        else:
            res = tuple([function(obj) for obj in objects_sub])
    else:
        res = tuple(objects_sub)
    
    if len(res) == 1:
        return res[0]
    else:
        return res
    

def n_random_resamples(*objects, samples, n_repeats, 
                       function=None, function_kwargs=None,
                       replace=True, with_dask=True):
    """
    Repeatedly randomly resample from provided xarray objects and return
    the results of the subsampled dataset passed through a provided function
                
    Parameters
    ----------
    objects : xarray DataArray or Dataset
        Objects containing data to be resampled.
        The coordinates of the first object are used for resampling and
        the same resampling is applied to all objects
    samples : dictionary
        Dictionary containing the dimensions to subsample, the number of samples and the continuous block size \
        within the sample. Of the form {'dim1': (n_samples, block_size), 'dim2': (n_samples, block_size)}
    n_repeats : int
        Number of times to repeat the resampling process
    function : function object, optional
        Function to reduced the subsampled data
    function_kwargs : dictionary, optional
        Keyword arguments to provide to function
    replace : boolean, optional
        Whether the sample is with or without replacement
    with_dask : boolean, optional
        If True, use dask to parallelize across n_repeats using dask.delayed
    write_to_disk
                
    Returns
    -------
    sample : xarray DataArray or Dataset
        Array containing the results of passing the subsampled data through function
    """

    if with_dask & (n_repeats > 1000):
        n_objects = itertools.repeat(objects[0], times=n_repeats)
        b = db.from_sequence(n_objects, npartitions=100)
        rs_list = b.map(random_resample, *(objects[1:]), 
                        **{'samples':samples, 'function':function, 
                           'function_kwargs':function_kwargs, 'replace':replace}).compute()
    else:              
        resample_ = dask.delayed(random_resample) if with_dask else random_resample
        rs_list = [resample_(*objects,
                             samples=samples,
                             function=function,
                             function_kwargs=function_kwargs,
                             replace=replace) for _ in range(n_repeats)] 
        if with_dask:
            rs_list = dask.compute(rs_list)[0]
    
    if len(objects) == 1:
        return xr.concat([r.unify_chunks() for r in rs_list], dim='k')
    else:
        return tuple([xr.concat([r.unify_chunks() for r in rs], dim='k') for rs in zip(*rs_list)])

In [ ]:
alpha = 5
n_repeats = 100
n_block = 1

fcst_moments = n_random_resamples(ds_stacked, 
                                  samples={'sample': (len(da_awap['time']), n_block)}, 
                                  n_repeats=n_repeats,
                                  function=get_first_four_moments,
                                  function_kwargs={'dim' : 'sample'})